In [ ]:
%%HTML
<style>
.container { width:100% }
</style>

In [ ]:
%run ./Muehle_Utilities.ipynb
import copy

## Spielphasen

Die Funktion `get_player_phase()` prüft in welcher Phase sich der übergebenen Spieler befindet. Hat er noch setzbare Steine, ist also sein Wert in `remaining` größer null, befindert er sich in Phase eins. Ist dies nicht der Fall und auch auf dem Feld befinden sich nur noch drei Steine, so befindet der Spieler sich in der Springphase. Ist dem nicht so, so ist der Spieler in Phase zwei, der Zugphase.
Die Eingabeparameter der Funktion sind ein Spielstatus, sowie ein Spieler in der Form 1 für Spieler eins und 2 für Spieler zwei. Zurückgegeben wird die Spielphase des übergebenen Spielers in dem übergebenen Spiel.

In [ ]:
def get_player_phase(state, player):
    [remaining, board] = state
    if remaining[player - 1] >= 1: 
        return 1
    elif count_player_pieces(state, player) == 3:
        return 3
    else:
        return 2

`possible_positions_phase_two()` berechnet die nächsten möglichen klickbaren Positionen, falls der Spieler in Phase 2 (Zugphase) ist. Dies sind ausgehend vom gewählten Stein alle benachbarten leeren Felder.
Hierfür wird ein Spielstatus, sowie die Position des ausgewählten Steines mit ring und cell übergeben. Zurückgegeben wird eine Liste der möglichen Position, auf die gezogen werden kann.

In [ ]:
def possible_positions_phase_two(state, ring, cell):
    neighboring = set(neighboring_positions((ring, cell)))
    empty = set(empty_positions(state))
    possible_positions = neighboring.intersection(empty)
    return list(possible_positions)

`next_positions()` berechnet in welcher Phase sich der Spieler befindet und dementsprechend, wo er als nächstes klicken kann. Falls der Spieler in Phase 1 (Setzphase) oder Phase 3 (Springphase) ist, dies sind unabhängig vom aktuellen Spieler oder in Phase 3 auch unabhängig vom ausgewählten Stein alle leeren Felder. Für Phase 2 wird die zuvor definierte Funktion `possible_positions_phase_two()` aufgerufen.
Übergeben wird der Funktion `next_positions()` hierfür ein Spielstatus, ein Spieler, und die Position des ausgewählten Steines (ring und cell).

In [ ]:
def next_positions(state, player, ring, cell):
    phase = get_player_phase(state, player)
    if phase == 1:
        return empty_positions(state)
    elif phase == 2:
        return possible_positions_phase_two(state, ring, cell)
    else:
        return empty_positions(state)

## next states

TODO: Dokumentation

In [ ]:
def next_states_phase_one(state, player):
    [remaining, board] = state
    boards = []
    # Alle aktuellen Mühlen finden
    mills = find_mills(board, player)
    
    # Ein Stein auf ein leeres Feld setzen
    empty = empty_positions(state)
    boards_after_placing = []
    for ring, cell in empty:
        new_board = copy.deepcopy(board)
        new_board[ring][cell] = player        
        boards_after_placing.append(new_board)
    
    # Für jedes Board überprüfen, ob neue Mühlen dazu gekommen sind und ggf. Steine entfernen
    for new_board in boards_after_placing:
        number_new_mills = count_new_mills(mills, find_mills(new_board, player))
        if number_new_mills > 0:
            boards.extend(beat_pieces(new_board, number_new_mills, player))
        else:
            boards.append(new_board)
    
    # Ein Stein aus den verbleibenden Steinen des Spielers entfernen
    remaining = [remaining[0] - 1, remaining[1]] if player == 1 else [remaining[0], remaining[1] - 1]
    
    # next_states zusammenbauen und zurückgeben
    return [[remaining, new_board] for new_board in boards]

In [ ]:
def next_states_phase_two(state, player):
    [remaining, board] = state
    boards = []
    # Alle aktuellen Mühlen finden
    mills = find_mills(board, player)
    
    # Alle Steine des Spielers als Startposition durchgehen und auf mögliche Nachbarpositionen verschieben
    pieces = player_pieces(state, player)
    boards_after_placing = []
    for ring_start, cell_start in pieces:
        positions = possible_positions_phase_two(state, ring_start, cell_start)
        for ring_goal, cell_goal in positions:
            new_board = copy.deepcopy(board)
            new_board[ring_start][cell_start] = 0    
            new_board[ring_goal][cell_goal] = player        
            boards_after_placing.append(new_board)
    
    # Für jedes Board überprüfen, ob neue Mühlen dazu gekommen sind und ggf. Steine entfernen
    for new_board in boards_after_placing:
        number_new_mills = count_new_mills(mills, find_mills(new_board, player))
        if number_new_mills > 0:
            boards.extend(beat_pieces(new_board, number_new_mills, player))
        else:
            boards.append(new_board)
    
    # next_states zusammenbauen und zurückgeben
    return [[remaining, new_board] for new_board in boards]

In [ ]:
def next_states_phase_three(state, player):
    [remaining, board] = state
    boards = []
    # Alle aktuellen Mühlen finden
    mills = find_mills(board, player)
    
    # Alle Steine des Spielers als Startposition durchgehen und auf leere Positionen verschieben
    pieces = player_pieces(state, player)
    empty = empty_positions(state)
    boards_after_placing = []
    for ring_start, cell_start in pieces:
        for ring_goal, cell_goal in empty:
            new_board = copy.deepcopy(board)
            new_board[ring_start][cell_start] = 0    
            new_board[ring_goal][cell_goal] = player        
            boards_after_placing.append(new_board)
    
    # Für jedes Board überprüfen, ob neue Mühlen dazu gekommen sind und ggf. Steine entfernen
    for new_board in boards_after_placing:
        number_new_mills = count_new_mills(mills, find_mills(new_board, player))
        if number_new_mills > 0:
            boards.extend(beat_pieces(new_board, number_new_mills, player))
        else:
            boards.append(new_board)
    
    # next_states zusammenbauen und zurückgeben
    return [[remaining, new_board] for new_board in boards]

In [ ]:
def next_states(state, player):
    phase =  get_player_phase(state, player)
    if phase == 1:
        return next_states_phase_one(state, player)
    elif phase == 2:
        return next_states_phase_two(state, player)
    else:
        return next_states_phase_three(state, player)

## Spielende

Die Funktion `finished()` prüft, ob das Spiel für den übergebenen Zustand zu Ende ist. 

Solange ein Spieler noch Steine setzen kann, ist das Spiel noch nicht entschieden. Ansonsten gibt es zwei Möglichkeiten, bei denen das Spiel noch nicht zu Ende ist. Die erste Situation tritt ein, wenn ein Spieler weniger als drei Steine hat. Er hat dann verloren. Bei der zweiten Option verliert ein Spieler, wenn er sich in Phase 2 befindet, aber nicht mehr ziehen kann. Dies kommt vor, wenn alle benachbarten Felder seiner Steine vom Gegenspieler besetzt sind.

In [ ]:
def finished(state):
    [remaining, board] = state
    # Wenn man noch Steine setzen kann, hat man nicht verloren
    if any(p > 0 for p in remaining): return False
    
    for player in [1, 2]:
        # Wenn man weniger als 3 Steine hat, hat man verloren
        if count_player_pieces(state, player) < 3: return True
        
        # Wenn man in Phase 2 ist und nicht mehr ziehen kann, aber in Phase 3 ist, hat man verloren
        # ToDo: später ersetzen durch:
        # if len(nextStates(state, player)) == 0 or len(nextStates(state, oponnent(player))) == 0: return True
        pieces = player_pieces(state, player)
        phase = get_player_phase(state, player)
        if not any(is_movable(state, position) for position in pieces) and phase == 2:
            return True

    # Falls die beiden Endsituationen für keinen Spieler eingetreten sind, ist das Spiel nicht zu Ende
    return False

Die Funktion `utility()` nimmt einen Zustand für den das Spiel beendet ist, also `finished(state)==True` und einen Spieler und gibt einen numerischen Wert zurück, der folgendermaßen zu interpretieren ist:
* -1 = Der Spieler hat verloren
* 0 =  Unentschieden
* 1 =  Der Spieler hat gewonnen

In [ ]:
def utility(terminal_state, player):
    state = terminal_state
    [remaining, board] = state
    utility = 0
    # Überprüfen, ob die Spieler weniger als drei Steine haben
    if count_player_pieces(state, player) < 3: utility = -1
    if count_player_pieces(state, opponent(player)) < 3: utility +=  1
        
    # utility = -1, wenn Spieler weniger als 3 Steine hat -> Ergebnis zurückgeben
    # utility = 0 wenn beide weniger als 3 Steine haben -> Prüfen, ob Spieler noch ziehen können
    # utility = 1, wenn Gegner weniger als 3 Steine hat -> Ergebnis zurückgeben
    if utility != 0: return utility
    
    # Wenn der Spieler in Phase 2 ist und nicht mehr ziehen kann, hat er verloren
    # ToDo: später ersetzen durch:
    # if len(nextStates(state, player)) == 0: utility = -1
    pieces = player_pieces(state, player)
    phase = get_player_phase(state, player)
    if not any(is_movable(state, position) for position in pieces) and phase == 2:
        utility = -1

    # Wenn der Gegner in Phase 2 ist und nicht mehr ziehen kann, hat der Spieler gewonnen
    # Es sei denn, das trifft auch auf den Spieler zu, dann handelt es sich um ein unentschieden
    # ToDo: später ersetzen durch:
    # if len(nextStates(state, opponent(player))) == 0: utility += 1
    pieces = player_pieces(state, opponent(player))
    phase = get_player_phase(state, opponent(player))
    if not any(is_movable(state, position) for position in pieces) and phase == 2:
        utility += 1
    
    return utility